In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
###  키워드 분석 클래스 시작
class KeywordAnalysis() :
    def __init__(self) :
        return None

In [6]:
# ! pip install customized_konlpy

In [7]:
# ! pip install git+https://github.com/ssut/py-hanspell.git

In [8]:
    ### 전처리 모듈
    def preproc(self, reviews, custom_stopwords, custom_noun, spellcheck=False) :

        print("전처리 시작")

        import pandas as pd
        import numpy as np
        import re
        from ckonlpy.tag import Twitter
        from hanspell import spell_checker
        from tqdm import tqdm

        #  문서군 변수 생성
        documents = []

        #  기존 불용어 위에 사용자 지정 불용어 적용
        with open("./data/한국어불용어_new.txt", "r", encoding="utf-8") as f :
            stop_words = f.read()
        f.close()
        stop_words = stop_words.split(",")
        stop_words.extend(custom_stopwords)

        #  사용자 지정 단어 사전 추가
        from ckonlpy.tag import Twitter
        custom_okt = Twitter()
        for n in custom_noun:
            custom_okt.add_dictionary(n, "Noun")

        #  결측치가 존재하는 record 제거
        reviews.dropna(inplace=True)

        #  의미있는한글(ㅋㅋ, ㅎㅎ이런거 말고), 알파벳, 숫자, 띄어쓰기 제외한 글자 삭제
        for idx, comment in enumerate(reviews["comment"]) :
            comment = re.sub(r"[^가-힣a-zA-Z0-9 ]", "", comment)
            reviews.iloc[idx,2] = comment

        #  띄어쓰기, 맞춤법 자동 교정 (주의!! 엄청 오래걸림!!!)
        if spellcheck == True :
            from hanspell import spell_checker
            for comment in tqdm(reviews["comment"]) :
                comment = spell_checker.check(comment).checked
                reviews.iloc[idx,2] = comment

        #  불용어 제거, 한글자 단어 제거
        for comment in reviews["comment"] :
            document = ""
            words = custom_okt.nouns(comment)
            for word in words :
                if (len(word) >= 2) or (word in custom_noun) :
                    if word not in stop_words :
                        document += word + " "
            document = document.rstrip()
            documents.append(document)

        print("전처리 완료, 문서군 생성\n")
        #  문서군 변수 리턴
        return documents

In [9]:
    ###  키워드 추출 모듈

    #  빈도수 기반 키워드를 추출하는 모듈
    def cnt(self, documents, filename="", cnt_th=10) :

        print("빈도수 기반 키워드 추출 시작")

        import pandas as pd
        import numpy as np
        from collections import Counter

        #  문서군 내의 모든 문서들을 하나의 문서로 모음
        documents_sum = []
        for document in documents :
            words = document.split(" ")
            if words[0] != "" :   # 공백이 카운트되는것 방지용
                documents_sum.append(words)
        documents_sum = sum(documents_sum, [])

        #  단어들의 빈도수 세기
        counter = Counter(documents_sum)
        vocab_sorted = sorted(counter.items(),  key=lambda x : x[1], reverse=True)

        #  빈도수 cnt_th 이상인 단어만 추출
        vocab_result = []
        for vocab in vocab_sorted :
            if vocab[1] >= cnt_th :
                vocab_result.append(vocab)

        #  변수 vocab_result를 데이터프레임으로 만들기
        data = np.array([None for _ in range(len(vocab_result)*2)])
        data = data.reshape((len(vocab_result),2))
        result_df = pd.DataFrame(data, columns=["noun", "count"])

        for idx, vocab in enumerate(vocab_result) :
            result_df.iloc[idx, 0] = vocab[0]
            result_df.iloc[idx, 1] = vocab[1]

        #  지정한 파일명으로 csv 파일 저장
        #  (파일명을 지정하지 않으면 파일로 저장되지 않고 리턴만 함.)
        if filename != "" :
            result_df.to_csv("{}.csv".format(filename), index=False, sep=",", encoding="utf-8")
            print("csv파일 저장 완료")

        print("빈도수 기반 키워드 추출 완료\n")
        return result_df

In [10]:

    #  TF-IDF 방식으로 키워드를 추출하는 모듈
    def tfidf(self, documents, filename="") :

        print("TF-IDF 기반 키워드 추출 시작")

        import pandas as pd
        import numpy as np
        from sklearn.feature_extraction.text import TfidfVectorizer 

        #  문서군의 TF-IDF 행렬 생성
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_mat = tfidf_vectorizer.fit_transform(documents)
        tfidf_mat = pd.DataFrame(
            tfidf_mat.toarray(), columns = tfidf_vectorizer.get_feature_names_out())

        #  각 문서별로 TF-IDF 값이 가장 높은 단어만 선별
        max_mat = pd.DataFrame(columns=["word", "tfidf"])
        max_mat["tfidf"] = tfidf_mat.max(axis="columns")
        max_mat["word"] = tfidf_mat.idxmax(axis="columns")

        #  각 단어들에 대한 TF-IDF값들 통계
        #  word : 단어
        #  freq_count : 해당 단어의 TF-IDF값이 가장 높게 나온 문서 개수
        #  tfidf_mean, tfidf_std : 해당 단어의 TF-IDF값들 평균, 표준편차
        statistic_mat = pd.DataFrame(
            columns=["word", "freq_count", "tfidf_mean", "tfidf_std"])

        th = 0.6  # TF-IDF값이 0.6 이상인 단어만 저장
        statistic_mat["word"] = list(
            max_mat[max_mat["tfidf"]>th].groupby("word")["word"].count().index)

        statistic_mat["freq_count"] = list(
            max_mat[max_mat["tfidf"]>th].groupby("word")["word"].count())

        statistic_mat["tfidf_mean"] = list(
            max_mat[max_mat["tfidf"]>th].groupby("word")["tfidf"].mean())

        statistic_mat["tfidf_std"] = list(
            max_mat[max_mat["tfidf"]>th].groupby("word")["tfidf"].std())

        statistic_mat.sort_values(by="freq_count", ascending=False, inplace=True)
        statistic_mat = statistic_mat.head(100)  # 빈도수 상위 100개만 저장
        statistic_mat.reset_index(drop=True, inplace=True)
        statistic_mat = round(statistic_mat, 4)

        if filename != "" :
            statistic_mat.to_csv("{}.csv".format(filename),index=False ,sep=",", encoding="utf-8")
            print("TF-IDF 기반 키워드 csv 파일 저장 완료")

        print("TF-IDF 기반 키워드 추출 완료\n")
        return statistic_mat

In [11]:
    ### 기타 분류 작업에 필요한 모듈    

    #  특정 키워드가 들어있는 문서만 골라내는 모듈
    def sel_doc(self, documents, keyword) :

        result_documents = []

        for document in documents :
            if keyword in document :
                result_documents.append(document)

        return result_documents


    #  빈도수 기반 키워드를 추출한것 중 빈도수가 가장 높게 나온것들 10개
    def cnt_top10(self, result_df) :

        top10_list = []

        import pandas as pd

        for idx, word in enumerate(result_df["noun"]) :
            if idx == 10 : break
            top10_list.append(word)
            # print(top10_list)

        return top10_list


In [12]:
# 키워드 분석 클래스 끝
# ======================================================================= #

In [13]:
###  시각화 클래스 시작
class Visualization() :
    #  키워드 추출 결과 csv파일들을 각종 방식으로 시각화하는 기능들이 들어있음.
    def __init__(self):
        return None

In [14]:
    #  워드클라우드
    def wordc(self, csvfile, filename="") :

        from wordcloud import WordCloud
        import pandas as pd
        import matplotlib
        import matplotlib.pyplot as plt

        matplotlib.rcParams['figure.figsize'] = (5,5)
        matplotlib.rcParams['font.family'] = "NanumGothic"
        matplotlib.rcParams['font.size'] = 10

        wc_source = dict()
        for idx in range(len(csvfile)) :
            if idx == 100 : break
            wc_source[csvfile.iloc[idx, 0]] = csvfile.iloc[idx, 1]

        wc = WordCloud(
            font_path="나눔고딕ExtraBold",width=400, height=400, scale=2.0, max_font_size=250,
            background_color="white", colormap="autumn", random_state = 20)
        gen = wc.generate_from_frequencies(wc_source)

        plt.title(f"{filename}")

        wc.to_file(f'시각화_wordcloud_{filename}.png')
        plt.imshow(gen)
        plt.show()
        return

In [15]:
    #  파이차트
    def pie(self, csvfile, filename="") :

        import pandas as pd
        import matplotlib
        import matplotlib.pyplot as plt

        matplotlib.rcParams['figure.figsize'] = (5,5)
        matplotlib.rcParams['font.family'] = "NanumGothic"
        matplotlib.rcParams['font.size'] = 10


        values = []
        labels = []

        th = 20   # 상위 20개만
        for idx in range(len(csvfile)) :
            values.append(csvfile.iloc[idx,1]) # count
            labels.append(csvfile.iloc[idx,0]) # 키워드
            if idx == th : break
        
        wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
        plt.pie(
            values, labels=labels,
            startangle=90, autopct='%.1f%%', counterclock=False, wedgeprops=wedgeprops)

        plt.title(f"{filename}")

        plt.savefig(f'시각화_piechart_{filename}.png')
        plt.show()

        return

In [16]:
    #  막대그래프
    def bar(self, csvfile, filename="", tfidf=False) :

        import pandas as pd
        import numpy as np
        import matplotlib
        import matplotlib.pyplot as plt

        matplotlib.rcParams['figure.figsize'] = (15,6)
        matplotlib.rcParams['font.family'] = "NanumGothic"
        matplotlib.rcParams['font.size'] = 10

        th = 40  # 상위 40개만
        x = np.arange(th)

        values = []
        labels = []

        for idx in range(len(csvfile)) :
            if idx == th : break
            values.append(csvfile.iloc[idx,1]) # count
            labels.append(csvfile.iloc[idx,0]) # 키워드

        plt.bar(
            labels, values,
            width=0.7)
        plt.xticks(rotation=45)

        plt.title(f"{filename}")
        plt.xlabel("키워드", labelpad=11)
        if tfidf :
            plt.ylabel("TF-IDF값이 가장 높았던 문서 개수", labelpad=11)
        else :
            plt.ylabel("전체 문서에서 등장한 회수", labelpad=11)

        plt.savefig(f'시각화_bargraph_{filename}.png')
        plt.show()

        return

In [17]:
# 시각화 클래스 끝
# ======================================================================= #

In [18]:
### 전역변수부 시작

#  내가 직접 지정한 불용어 리스트 작성
custom_stopwords = [
    "범죄도시", "범죄 도시", "범죄", "도시", "죽지",
    "늑대사냥", "늑대", "사냥",
    "공조", "인터네셔날","인터내셔날",
    "한산", "용", "용의", "출현", "용의출현", "장군님", "장군",
    "뜨거운피", "피", "뜨거운",
    "외계인", "외계", "인",
    "영화", "무비", "영화관", "한국영화",
    "진짜", "진자", "완전", "존나", "졸라", "느낌", "뭔가", "제일", "개", "보고", "사람", "보지",
    "어요", "오늘", "그냥", "생각", "면서", "더니", "인적", "거지", "보기", "나름", "살짝",
    "정말", "대박", "역대", "최고", "어제", "편이", "계속", "요소", "처럼", "이나", "역시", "부분", "던데",
    "스포", "개봉", "한번", "내내", "구나", "때문", "어서", "정도", "다가", "다시", "누가", "덕분", "항상",
    "봤는데", "왔는데",
    "스포일러가 포함된 감상평입니다."
]

#  ckonlpy에 추가할 단어 지정
custom_noun = [

    #  [ 범죄도시 ]
    "장이수", "강해상", "손석구", "마석도", "마동석", "마블리", "박지환", "윤계상",
    "2부", "2탄", "2편", "1부", "1탄", "1편",
    #  * 같은 단어 : 장이수=이수, 마석도=마동석=마블리
    #  * 불용어 처리하면 안되는 단어 :
    #  - 구씨 : 배우 손석구가 다른 작품에서 맡았던 케릭터 이름

    #  [ 늑대사냥 ]
    "서인국", "장동윤", "박호산", "정소민", "고창석", "성동일",
    "콘에어", "콘 에어", "잔인",
    #  * 같은 단어 : 콘에어=콘 에어
    #  * 불용어 처리하면 안되는 단어 :
    #  - 콘에어 : 니콜라스 케이지 출연 1997년 영화. 늑대사냥과 비슷한 소재를 다루고있음.

    #  [ 공조2 ]
    "현빈", "유해진", "윤아", "다니엘 헤니", "다니엘헤니", "다니엘", "헤니", "진선규",
    "장영남", "박훈", "임성재", "림청렬", "청렬", "강진태", "진태", "장명준", "박소연",
    "박상위", "박민영",
    "불시착", "사랑의 불시착",
    # 같은 단어 : 다니엘헤니=다니엘=헤니, 불시착=사랑의 불시착

    #  [ 외계인1부 ]
    "류준열", "김우빈", "김태리", "소지섭", "염정아", "조우진", "이하늬",
    "무륵", "김현중", "이안", "문도석", "흑설", "청운",
    "2부", "2탄", "2편", "1부", "1탄", "1편", "돈", "SF", "sf", "SF물", "sf물",
    #  * 불용어 처리하면 안되는 단어 :
    #  - 돈 : 한 글자 '돈 아깝다'같은 말이 종종 보임.
    #  - SF, sf : 장르명. 한글이 아니라고해서 없애면 안됨.

    #  [ 한산 ]
    "박해일", "안성기", "변요한", "손현주", "김성규", "김성균", "김향기", "택연", "옥택연",
    "이순신", "장군님", "장군", "어영담", "원균", "준산", "히데요시", "도요토미", "도요토미 히데요시",
    "한산도", "대첩", "한산도 대첩", "임진왜란", "유키나가", "학익진",
    #  * 같은 단어 : 이순신=이순신장군
    #  * 불용어 처리하면 안되는 단어 :
    #  - 하라 : '전군 출정하라', '선회하라', '발포하라'가 이 영화의 명대사인가봄.

    # 뜨거운피 배우명/등장인물명
    "정우", "김갑수", "최무성", "지승현", "김해곤", "윤지혜", "이홍내", "정호빈"
    #  * 불용어 처리하면 안되는 단어 :
    #  - 짱구 : 배우 정우가 다른 작품에서 맡았던 케릭터 이름

    ]

#  클래스 생성자 만들기
keywordAnalysis = KeywordAnalysis()
visualization = Visualization()

### 전역변수부 끝
# ======================================================================= #

In [19]:
### 메인부 시작
if __name__ == "__main__" :

    import pandas as pd
    from glob import glob
    import re

    ### 유튜브 댓글
    file_names = glob("./result/크롤링_유튜브댓글/*.csv")
    for file_name in file_names :
        reviews = pd.read_csv(file_name, encoding='utf-8', sep=",")

        #  전처리
        documents = keywordAnalysis.preproc(reviews, custom_stopwords, custom_noun, spellcheck=False)
        file_name = re.sub(r".*\\", "", file_name)  
        file_name = re.sub(r"\.csv", "", file_name)  # 파일 저장하기 편하려고...

        #  빈도수 기반 키워드 추출
        if file_name == "뜨거운피_예고편" :
            cnt_th = 4  # 빈도수 4 이상만 추출
        else :
            cnt_th = 10  # 빈도수 10 이상만 추출
        result_cnt = keywordAnalysis.cnt(documents, filename=f"단어빈도수_유튜브_{file_name}", cnt_th=cnt_th)


        ### 이건 연구중...
        
        # #  그 중에서 빈도수 top10 키워드 목록 생성
        # top10 = keywordAnalysis.cnt_top10(result_cnt)

        # result_cnt["high_TFIDF_keywords"] = [None for _ in range(len(result_cnt))]
        # for keyword_cnt in top10 :
        #     selected_docs = keywordAnalysis.sel_doc(documents, keyword_cnt)
        #     result_tfidf = keywordAnalysis.tfidf(selected_docs)
        #     result_tfidf = result_tfidf.head(5)

        #     high_TFIDF_keywords = {}
        #     for idx in range(len(result_tfidf)) :
        #         high_TFIDF_keywords[result_tfidf.iloc[idx, 0]] = result_tfidf.iloc[idx, 2]

        #     result_cnt.loc[keyword_cnt, "high_TFIDF_keywords"] = high_TFIDF_keywords
        
        # result_cnt.to_csv(f"test_단어별TFIDF포함_{file_name}")

In [20]:
    ### 네이버 영화

    #  파일 한꺼번에 불러오기
    file_names = glob("./result/크롤링_네이버영화/*.csv")

    for file_name in file_names :
        reviews = pd.read_csv(file_name, encoding='utf-8', sep=",")

        #  전처리
        documents = keywordAnalysis.preproc(reviews, custom_stopwords, custom_noun, spellcheck=False)
        file_name = re.sub(r".*\\", "", file_name)  # 파일 저장하기 편하려고...
        file_name = re.sub(r"csv", "", file_name)  # 파일 저장하기 편하려고...

        #  빈도수 기반 키워드 추출
        cnt_th = 10
        result_cnt_all = keywordAnalysis.cnt(documents, filename=f"단어빈도수_네이버_{file_name}_전체", cnt_th = cnt_th)    


        ### 이건 연구중...
        # #  빈도수 기반 키워드 추출 (높은 평점)
        # result_cnt_hscore = keywordAnalysis.cnt(
        #     documents[documents["score"]>=8], f"단어빈도수_{file_name}_평점8점이상")    
        # #  빈도수 기반 키워드 추출 (낮은 평점)
        # result_cnt_lscore = keywordAnalysis.cnt(
        #     documents[documents["score"]<=4], f"단어빈도수_{file_name}_평점4점이하")    


In [ ]:
### 메인부 끝
# ======================================================================= #

In [21]:
print(cnt(a, filename='늑대사냥'))

NameError: name 'a' is not defined

In [22]:
import Word2Vec

ModuleNotFoundError: No module named 'Word2Vec'